# Multi-Agent Collaboration (MAC) Example: Hedge Fund Assistant
---

This example demonstrates a Hedge Fund Assistant using MAC principles with the following structure:

- A **Supervisor Agent** powered by the Amazon Nova Lite foundation model routes user queries.

- Three **Sub-Agents**:
    - **Fundamental Analyst Agent** (uses Claude 3 Haiku as the FM)
    - **Technical Analyst Agent** (uses Amazon Nova Lite as the FM)
    - **Market Analyst Agent** (uses Claude 3 Haiku as the FM)

Each sub-agent has access to a set of tools to fetch data, analyze it, and provide actionable insights.

#### Set a logger

The Strands Agents SDK implements a straightforward logging approach:

1. **Module-level Loggers**: Each module in the SDK creates its own logger using logging.getLogger(__name__), following Python best practices for hierarchical logging.

2. **Root Logger**: All loggers in the SDK are children of the "strands" root logger, making it easy to configure logging for the entire SDK.

3. **Default Behavior**: By default, the SDK doesn't configure any handlers or log levels, allowing you to integrate it with your application's logging configuration.

In [ ]:
# import logging and set a logger for strands
import logging
# import the strands agents and strands tools that we will be using
from strands import Agent
from strands_tools import swarm

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

In [ ]:
# install other requirements
import os
import sys
import json
import time
import boto3
import shutil
import logging
import zipfile
import subprocess
from dotenv import load_dotenv


# Load the environment variables that are defined in the ".env" file. This contains the 
# financial data API key that will enable the user to access the data
load_dotenv()
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."
# os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region (default)
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

### Load config file
---

This [config](config.yaml) file is a `yaml` file that contains information that this solution uses, including the result files, the model information for each of the agent used in this multi agentic system, inference parameters, and more.

In [ ]:
!uv pip install PyYAML aiohttp openai
from utils import load_config

# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the config file
config_data = load_config('config.yaml')
logger.info(f"Loaded config from local file system: {json.dumps(config_data, indent=2)}")

### Define tools
---

This example creates a supervisor agent and integrates the three sub-agents, each having specific tools for financial data analysis:

1. **Fundamental Analyst Agent**:
    - Tools: Retrieve income statements, balance sheets, and cash flow statements.

2. **Technical Analyst Agent**:
    - Tools: Fetch stock prices, current prices, and compute technical indicators (e.g., `RSI`, `MACD`, `SMA`).

3. **Market Analyst Agent**:
    - Tools: Access options chain data, insider trading information, and market news.

In [ ]:
api_key = os.environ.get("FINANCIAL_DATASET_API")
if api_key is not None:
    print("Financial Data API key found in environment variables.")
else:
    print(f"API key not found, enter it in the .env file.")

In [ ]:
import os
import uuid
import logging
from datetime import datetime
from typing import Dict, Any, Optional

tool_use_ids=[]

# ─── LOGGER SETUP ──────────────────────────────────────────────────────────────
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def comprehensive_callback_handler(**kwargs):
    """
    Enhanced comprehensive callback handler with LangSmith integration
    """
    
    # === REASONING EVENTS (Agent's thinking process) ===
    if kwargs.get("reasoning", False):
        if "reasoningText" in kwargs:
            reasoning_text = kwargs['reasoningText']
            logger.info(f"🧠 REASONING: {reasoning_text}")
            
        if "reasoning_signature" in kwargs:
            logger.info(f"🔍 REASONING SIGNATURE: {kwargs['reasoning_signature']}")
    
    # === TEXT GENERATION EVENTS ===
    elif "data" in kwargs:
        # Log streamed text chunks from the model
        logger.info(kwargs["data"], end="")
        if kwargs.get("complete", False):
            logger.info("")  # Add newline when complete
    
    # === TOOL EVENTS ===
    elif "current_tool_use" in kwargs:
        tool = kwargs["current_tool_use"]
        tool_use_id = tool["toolUseId"]
        
        if tool_use_id not in tool_use_ids:
            tool_name = tool.get('name', 'unknown_tool')
            tool_input = tool.get('input', {})
            
            logger.info(f"\n🔧 USING TOOL: {tool_name}")
            if "input" in tool:
                logger.info(f"📥 TOOL INPUT: {tool_input}")
            tool_use_ids.append(tool_use_id)
    
    # === TOOL RESULTS ===
    elif "tool_result" in kwargs:
        tool_result = kwargs["tool_result"]
        tool_use_id = tool_result.get("toolUseId")
        result_content = tool_result.get("content", [])
        
        logger.info(f"📤 TOOL RESULT: {result_content}")
    
    # === LIFECYCLE EVENTS ===
    elif kwargs.get("init_event_loop", False):
        logger.info("🔄 Event loop initialized")
        
    elif kwargs.get("start_event_loop", False):
        logger.info("▶️ Event loop cycle starting")
        
    elif kwargs.get("start", False):
        logger.info("📝 New cycle started")
        
    elif kwargs.get("complete", False):
        logger.info("✅ Cycle completed")
        
    elif kwargs.get("force_stop", False):
        reason = kwargs.get("force_stop_reason", "unknown reason")
        logger.info(f"🛑 Event loop force-stopped: {reason}")
    
    # === MESSAGE EVENTS ===
    elif "message" in kwargs:
        message = kwargs["message"]
        role = message.get("role", "unknown")
        logger.info(f"📬 New message created: {role}")
    
    # === ERROR EVENTS ===
    elif "error" in kwargs:
        error_info = kwargs["error"]
        logger.error(f"❌ ERROR: {error_info}")

    # === RAW EVENTS (for debugging) ===
    elif "event" in kwargs:
        # Log raw events from the model stream (optional, can be verbose)
        logger.debug(f"🔍 RAW EVENT: {kwargs['event']}")
    
    # === DELTA EVENTS ===
    elif "delta" in kwargs:
        # Raw delta content from the model
        logger.debug(f"📊 DELTA: {kwargs['delta']}")
    
    # === CATCH-ALL FOR DEBUGGING ===
    else:
        # Log any other events we might have missed
        logger.debug(f"❓ OTHER EVENT: {kwargs}")


In [ ]:
# define the tools for the finance agent
# Strands Agents SDK makes it straightforward to 
# turn Python functions into callable tools within an AI agent.

# transform any Python function into a tool simply by adding the @tool decorator—its 
# docstring and type hints automatically generate the tool's specification
import os
import requests
from strands import tool
from typing import Dict, List

@tool
def get_income_statements(
    ticker: str, 
    period: str = "ttm", 
    limit: int = 10
) -> Dict:
    """
    Get income statements for a ticker. This is one of the functions that is used to get the 
    income statements based on the ticker specified by the user
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period (str, optional): Time period for the data. Options:
            - 'ttm': Trailing twelve months (default)
            - 'annual': Annual data
            - 'quarterly': Quarterly data
        limit (int, optional): Maximum number of statements to return (default: 10)
        
    Returns:
        Dict: JSON response containing income statement data
        
    Raises:
        RuntimeError: If API key is missing or API request fails
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = (
        f'https://api.financialdatasets.ai/financials/income-statements'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        print(f"OUTPUT FROM THE GET_INCOME_STATEMENTS TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "income_statements": [], "error": str(e)}

@tool
def get_balance_sheets(
    ticker: str, 
    period: str = "ttm", 
    limit: int = 10
) -> Dict:
    """
    Get balance sheets for a ticker and the specified limit and time period
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period (str, optional): Time period for the data. Options:
            - 'ttm': Trailing twelve months (default)
            - 'annual': Annual data
            - 'quarterly': Quarterly data
        limit (int, optional): Maximum number of balance sheets to return (default: 10)
        
    Returns:
        Dict: JSON response containing balance sheet data
        
    Raises:
        RuntimeError: If API key is missing or API request fails
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = (
        f'https://api.financialdatasets.ai/financials/balance-sheets'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        print(f"OUTPUT FROM THE GET_BALANCE_SHEETS TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "balance_sheets": [], "error": str(e)}

@tool
def get_cash_flow_statements(
    ticker: str, 
    period: str = "ttm", 
    limit: int = 10
) -> Dict:
    """
    Get cash flow statements for a ticker
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period (str, optional): Time period for the data. Options:
            - 'ttm': Trailing twelve months (default)
            - 'annual': Annual data
            - 'quarterly': Quarterly data
        limit (int, optional): Maximum number of cash flow statements to return (default: 10)
        
    Returns:
        Dict: JSON response containing cash flow data
        
    Raises:
        RuntimeError: If API key is missing or API request fails
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = (
        f'https://api.financialdatasets.ai/financials/cash-flow-statements'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        # fetch and return the output of the api call in the response
        print(f"OUTPUT FROM THE GET_CASH_FLOW_STATEMENTS TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "cash_flow_statements": [], "error": str(e)}

#### Create a custom python code execution tool
---

In this section, we will create a custom tool that lets you configure or use a model of choice to generate code based on the content and the user question and then execute that tool as well.

In [ ]:
# Represents the code execution prompt for the LLM
CODE_EXEC_PROMPT: str = """You are a Python code generator. Generate clean, efficient, and safe Python code based on the user's requirements.

Guidelines:
1. Only generate executable Python code
2. Include necessary imports at the top
3. Add comments to explain complex logic
4. Handle potential errors gracefully
5. Keep code concise but readable
6. Do not include any harmful or system-modifying operations
7. Focus on data processing, calculations, and analysis tasks
8. If working with data, assume it might be passed as context
9. Print results using print() statements so they appear in output

Always adhere to the user question and generate python code based on what the user is asking.

Return ONLY the Python code, no explanations or markdown formatting."""

In [ ]:
!uv pip install tiktoken tokenizers jinja2

In [ ]:
!uv pip show litellm


In [ ]:
import os
import subprocess
import tempfile
import sys
import time
from typing import Optional, Dict, Any, List
from strands import tool
from litellm import completion

### Define the system prompt for the financial agent
---

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# This is the system prompt that will be used for the financial agent
financial_agent_instruction = """You are a comprehensive fundamental analyst assistant that helps users analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements.
You require the user to provide a stock ticker symbol to analyze the company's financials.

You can perform the following types of analysis by calling the functions below:

1. Income Statement Analysis:
   - Revenue growth, gross profit, operating income, net income, earnings per share, and revenue/expense breakdown

2. Balance Sheet Analysis:
   - Asset composition (current and non-current), liabilities (current and non-current), shareholders' equity, retained earnings, and financial ratios (current ratio, debt-to-equity)

3. Cash Flow Statement Analysis:
   - Net cash flow from operations, capital expenditures, business acquisitions, issuance/repayment of debt, dividends, and changes in cash and equivalents

IMPORTANT: Always use the financial dataset API you have access to to call these functions and retrieve the data to answer the user question

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above. Always give information on where you got the data to answer the user question
and do not redact anything in your response.

Also, use the python repl tool function at the very end to generate report and analysis based on the context fetched so far and store
that in a file or provide that as an output.
"""
print(f"Going to use the financial agent system prompt: {financial_agent_instruction}")

In [ ]:
# Get the model information for the finance agent
finance_agent_model_info: Dict = config_data['model_information'].get('finance_agent_model_info')
print(f"Fetched the agent model information: {finance_agent_model_info}")

In [ ]:
# initialize the model that will power the financial agent
# in this case, we will use the claude 3-7 model to power the financial 
# agent
from strands.models import BedrockModel

# Define the current aws region
region: str = boto3.Session().region_name
print(f"Going to use the agent in the region: {region}")

# Create a bedrock model using the BedrockModel interface
bedrock_model = BedrockModel(
    model_id=finance_agent_model_info.get('model_id'),
    region_name=region,
    temperature=finance_agent_model_info['inference_parameters'].get('temperature'),
)
print(f"Initialized the bedrock model for the finance agent: {bedrock_model}")

### Create the financial agent
---

Next, we will simply create the finance agent. We will use a Callback handler function in this case. We will implement the custom callback function. This will be invoked at various points throughout the financial agent's lifecycle. 

Here is an example that captures streamed data from the agent and logs instead of printing:

### What are Callback handlers in Strands SDK?

Callback handlers are a powerful feature for Strands SDK that enables the user to intercept and process the events as they are occurring during the agent execution process. This means that the agent will be able to handle real time monitoring, custom output formatting and integration with other external systems.

This handling is done and can occur throughout the agent's lifecycle and includes:

1. The text generation process that is done with the model that underlies the agent. If the agent is generating text then the callback handler will catch that and emit it out during the agent execution process.

2. If the agent is using tools or has tools associated with it, then the agent will be using that and executing that. As a part of that process, the callback handler can catch those traces and emit it out for real time processing.

3. Reasoning process: If you want to handle the traces or track the reasoning process of the agent based on the LLM thinking to debug and optimize performance, that is possible through callback handlers as well.

4. Errors and completions: If there are any errors or completions during the process we can handle that using the callback handler functionality as well.

For more information on what you can log with Callback handler events, view the documentation [here](https://strandsagents.com/0.1.x/user-guide/concepts/streaming/callback-handlers/).

In [ ]:
# we will now set up logging for the agent using langfuse
import base64
otel_host = os.environ.get("LANGFUSE_HOST")
if otel_host:
    # Set up endpoint for OpenTelemetry
    otel_endpoint = str(os.environ.get("LANGFUSE_HOST")) + "/api/public/otel/v1/traces"

    # Create authentication token for OpenTelemetry
    auth_token = base64.b64encode(
        f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
    ).decode()
    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = otel_endpoint
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {auth_token}"


In [ ]:
# import the prebuilt tools.
# Calculator: Perform mathematical operations
# file_read: Read and parse files
# mem0_memory: Tool for managing memories using Mem0 (store, delete, list, get, and retrieve)
# python_repl: Run Python code
# current_time: Get the current date and time
# journal: Create structured tasks and logs for agents to manage and work from
from strands_tools import calculator, file_read, mem0_memory, python_repl, current_time, journal, python_repl

# Generate a unique session ID using UUID
session_id = str(uuid.uuid4())

# Create the financial agent
finance_agent = Agent(
    # this is the system prompt to the strands agent
    system_prompt=financial_agent_instruction, 
    # use the tools from the ones defined above for the 
    # finance agent
    tools = [
             # These are the custom built tools that the finance 
             # agent will use
             get_income_statements, 
             get_balance_sheets,
             get_cash_flow_statements, 
             # These are the prebuilt set of tools that the 
             # strands sdk agent already offers
             calculator, 
             python_repl, 
             file_read, 
             mem0_memory, 
             current_time, 
             journal],
    # define the callback handler in this case
    # This callback handler logs the reasoning, tool, lifecycle, 
    # raw, delta and message events.
    callback_handler=comprehensive_callback_handler, 
    trace_attributes={
            "session.id": session_id,  # Use UUID for unique session tracking
            "user.id": "agent-builder@strandsagents.com",
            "langfuse.tags": [
                "Strands-Agents-Builder",
            ],
        }
)

In [ ]:
# view some of the tools and the model through the agent config
finance_agent.model

In [ ]:
# view the details of the tools in a more human readable manner
def print_tools_summary(config):
    """Print a summary of all tools"""
    tools = config.get('tools', [])
    
    print("=" * 60)
    print(f"AGENT TOOLS SUMMARY")
    print("=" * 60)
    print(f"Total number of tools: {len(tools)}")
    print()
    
    for i, tool in enumerate(tools, 1):
        tool_spec = tool.get('toolSpec', {})
        name = tool_spec.get('name', 'Unknown')
        description = tool_spec.get('description', 'No description available')
        
        # Extract the first line of description for summary
        first_line = description.split('\n')[0] if description else 'No description'
        
        print(f"{i}. {name}")
        print(f"   Summary: {first_line}")
        print()

In [ ]:
# view the tools that the finance agent has access to
print_tools_summary(finance_agent.tool_config)

Strands Agents SDK provides support for asynchronous iterators through the stream_async method, enabling real-time streaming of agent responses in asynchronous environments like web servers, APIs, and other async applications.

In [ ]:
# Define the async function
async def process_streaming_response(agent, query):
    agent_stream = agent.stream_async(query)
    async for event in agent_stream:
        # Print only the text content, not the raw event data
        if isinstance(event, dict):
            # Check for 'data' field first (seems to contain the text)
            if 'data' in event:
                print(event['data'], end='', flush=True)
            # Fallback to nested event structure
            elif 'event' in event and 'contentBlockDelta' in event['event']:
                delta = event['event']['contentBlockDelta']['delta']
                if 'text' in delta:
                    print(delta['text'], end='', flush=True)
        else:
            # If it's not a dict, just print it as is
            print(event, end='', flush=True)
    print()  # Add a newline at the end


In [ ]:
import asyncio

# Define the query
query: str = """
I want you to perform a comprehensive financial analysis of AAPL. Please:

1. Retrieve and analyze the latest income statements, balance sheets, and cash flow statements for AAPL, 
if your tool encounters an error, then call the function again with the correct parameters.
2. Calculate key financial pointers from the data. Based on your output from the latency input statements, balance sheets and cash flow statements, 
generate code to create charts, pie charts, bar charts, execute mermaid diagrams to show interesting take aways from the financial data that you 
have in your context.
3. Identify trends over the past 3-5 years in revenue growth, profit margins, and cash generation
4. Assess Apple's financial health and liquidity position
5. Provide investment insights based on the financial data analysis
6. Store the key findings in memory for future reference
7. Create a journal entry summarizing today's AAPL analysis with actionable insights

Please present the analysis in a structured format with clear sections for each financial statement review, ratio analysis, trend identification, and final recommendations.
"""

In [ ]:
# simply run the agent to see the outputs
finance_agent(query)

### View the traces in the LangFuse dashboard
---

![trace1](img/trace1.png)

![trace2](img/trace2.png)

## Technical Analyst Agent
---

In this section, we will be crating the second sub agent that will be responsible for technical analysis to user questions. This inclues the following:

1. Questions about getting stock prices, i.e., based on a user question the ticker will go over a given data range and interval and the agent will get the stock prices.

2. Questions about seeking information on the current stock price - if the user asks for the current stock price, then the agent will call a tool to get the latest or current stock price for a given ticker.

3. Last, we will create a human in the loop for calculating technical indicators for a given time period.

In [ ]:
technical_agent_analysis_prompt: str = """
You are a technical analysis assistant that helps users analyze stock price movements and technical indicators. You can calculate and interpret various technical indicators using historical price data.

You have access to the following technical analysis capabilities based on the API key you have access to to call the available functions and get the following information:

1. Stock Price Data:
   - Current stock prices
   - Historical price data with various intervals
   - Custom date range analysis

2. Technical Indicators:
   - RSI
   - MACD
   - SMA
   - EMA
   - Bollinger Bands

You require the user to provide:
1. A stock ticker symbol
2. The type of technical indicator they want to analyze
3. Optionally: specific time periods or date ranges

Available functions:
1. get_current_stock_price: Retrieve latest stock price
2. get_stock_prices: Get historical price data for a specified period
3. get_technical_indicators: Calculate technical indicators for analysis

If you do not have access to the data that the user is asking for, do not make up an answer. Be completely accurate and only provide analysis based on the available technical indicators and price data.

Only answer questions related to technical analysis and price data based on the provided functions. If unsure, acknowledge limitations"""

In [ ]:
from typing import Union
import datetime

@tool
def get_stock_prices(ticker: str, start_date: str, end_date: str, limit: int = 5000) -> Union[Dict, str]:
    """
    Get historical stock prices for a ticker within a date range.
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        start_date (str): Start date in YYYY-MM-DD format
        end_date (str): End date in YYYY-MM-DD format
        limit (int): Maximum number of price records to return (default: 5000)
        
    Returns:
        Union[Dict, str]: JSON response containing historical price data or error message
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    logger.info(f"API Key present: {'yes' if api_key else 'no'}")
    if not api_key:
        logger.error("Missing FINANCIAL_DATASET_API environment variable")
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}
    url = (
        f"https://api.financialdatasets.ai/prices"
        f"?ticker={ticker}"
        f"&start_date={start_date}"
        f"&end_date={end_date}"
        f"&interval=day"
        f"&interval_multiplier=1"
        f"&limit={limit}"
    )
    try:
        logger.info(f"Making API request to: {url}")
        response = requests.get(url, headers={'X-API-Key': api_key})
        logger.info(f"API response status code: {response.status_code}")
        if response.status_code != 200:
            logger.error(f"API error: {response.text}")
            return {"error": f"API returned status code {response.status_code}"}
        print(f"RESPONSE FROM THE GET_STOCK_PRICES TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        logger.error(f"Error in get_stock_prices: {str(e)}", exc_info=True)
        return {"ticker": ticker, "prices": [], "error": str(e)}


@tool
def get_current_stock_price(ticker: str) -> Union[Dict, str]:
    """
    Get current stock price based on the ticker provided by the user
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        
    Returns:
        Union[Dict, str]: JSON response containing current price data or error message
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = f"https://api.financialdatasets.ai/prices/snapshot?ticker={ticker}"

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        print(f"RESPONSE FROM THE GET_CURRENT_STOCK_PRICES TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "price": None, "error": str(e)}

In [ ]:
from datetime import datetime, timedelta
from typing import Optional, Dict, Union

@tool
def get_technical_indicators(ticker: str, indicator: str, period: int = 14,
                           start_date: Optional[str] = None, end_date: Optional[str] = None) -> Union[Dict, str]:
    """
    This function calculates technical indicators based on the provided parameters.
    It supports the following indicators: SMA, EMA, and RSI.
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        indicator (str): Technical indicator to calculate ('SMA', 'EMA', or 'RSI')
        period (int): Period for the indicator calculation (default: 14)
        start_date (Optional[str]): Start date in YYYY-MM-DD format
        end_date (Optional[str]): End date in YYYY-MM-DD format
        
    Returns:
        Union[Dict, str]: JSON response containing technical indicator data or error message
    """
    try:
        # Get extra data for calculations by extending the start date
        adjusted_start = (datetime.strptime(start_date, "%Y-%m-%d") - timedelta(days=period * 2)).strftime("%Y-%m-%d")
        
        # Fetch price data
        price_data = get_stock_prices(
            ticker=ticker,
            start_date=adjusted_start,
            end_date=end_date,
            limit=5000
        )

        if "error" in price_data:
            return price_data

        # Process the price data
        prices = price_data["prices"]
        
        # Parse datetime more robustly - handle different formats
        for price in prices:
            time_str = price['time']
            # Remove timezone info and handle different formats
            if 'Z' in time_str:
                time_str = time_str.replace('Z', '')
            if 'T' in time_str:
                # ISO format: 2025-04-23T04:00:00
                price['datetime'] = datetime.fromisoformat(time_str)
            else:
                # Handle other formats if needed
                time_str = time_str.split(' EDT')[0].split(' EST')[0]
                price['datetime'] = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")

        # Sort by datetime to ensure chronological order
        prices.sort(key=lambda x: x['datetime'])

        # Filter to requested date range
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)  # Include end date
        filtered_prices = [p for p in prices if start_dt <= p['datetime'] < end_dt]

        if len(filtered_prices) < period:
            return {"error": f"Not enough data points. Need at least {period} days, got {len(filtered_prices)}"}

        result = {
            "ticker": ticker,
            "indicator": indicator.upper(),
            "period": period,
            "data": []
        }

        # Calculate indicators
        if indicator.lower() == "sma":
            # Simple Moving Average
            for i in range(period - 1, len(prices)):
                if prices[i]['datetime'] >= start_dt:
                    sma_prices = [prices[j]['close'] for j in range(i - period + 1, i + 1)]
                    sma_value = sum(sma_prices) / period
                    
                    result["data"].append({
                        "time": prices[i]['datetime'].strftime("%Y-%m-%d %H:%M:%S"),
                        "time_milliseconds": int(prices[i]['datetime'].timestamp() * 1000),
                        "value": round(float(sma_value), 2)
                    })
        
        elif indicator.lower() == "ema":
            # Exponential Moving Average
            multiplier = 2 / (period + 1)
            ema = prices[0]['close']  # Start with first price
            
            for i, price in enumerate(prices):
                if i == 0:
                    ema = price['close']
                else:
                    ema = (price['close'] * multiplier) + (ema * (1 - multiplier))
                
                if price['datetime'] >= start_dt:
                    result["data"].append({
                        "time": price['datetime'].strftime("%Y-%m-%d %H:%M:%S"),
                        "time_milliseconds": int(price['datetime'].timestamp() * 1000),
                        "value": round(float(ema), 2)
                    })

        elif indicator.lower() == "rsi":
            # Relative Strength Index
            if len(prices) < period + 1:
                return {"error": f"Not enough data for RSI calculation. Need at least {period + 1} days"}
            
            # Calculate price changes
            changes = []
            for i in range(1, len(prices)):
                changes.append(prices[i]['close'] - prices[i-1]['close'])
            
            # Calculate RSI for each day
            for i in range(period - 1, len(changes)):
                if prices[i + 1]['datetime'] >= start_dt:
                    # Get the last 'period' changes
                    period_changes = changes[i - period + 1:i + 1]
                    
                    gains = [max(change, 0) for change in period_changes]
                    losses = [abs(min(change, 0)) for change in period_changes]
                    
                    avg_gain = sum(gains) / period
                    avg_loss = sum(losses) / period
                    
                    if avg_loss == 0:
                        rsi = 100.0
                    else:
                        rs = avg_gain / avg_loss
                        rsi = 100 - (100 / (1 + rs))
                    
                    result["data"].append({
                        "time": prices[i + 1]['datetime'].strftime("%Y-%m-%d %H:%M:%S"),
                        "time_milliseconds": int(prices[i + 1]['datetime'].timestamp() * 1000),
                        "value": round(float(rsi), 2)
                    })
        
        else:
            return {"error": f"Unsupported indicator: {indicator}. Supported: SMA, EMA, RSI"}

        print(f"RESPONSE FROM THE GET_TECHNICAL_INDICATORS TOOL: {result}")
        return result
        
    except Exception as e:
        error_msg = f"Error in get_technical_indicators: {str(e)}"
        logger.error(error_msg)
        return {"error": error_msg}

In [ ]:
# let's create a list of tools for the technical analyst agent
technical_tools = [get_stock_prices, get_current_stock_price, get_technical_indicators]

In [ ]:
# define the model to be used by the technical analyst agent
technical_agent_model_info: Dict = config_data['model_information']['technical_agent_model_info']

In [ ]:
technical_agent = Agent(
    system_prompt = technical_agent_analysis_prompt, 
    tools = [
        current_time
    ] + technical_tools,
    model = BedrockModel(
    model_id=technical_agent_model_info.get('model_id'), 
    max_tokens=technical_agent_model_info.get('max_tokens')
    )
)

In [ ]:
print_tools_summary(technical_agent.tool_config)

In [ ]:
query: str = """
What is the current time and then based on that, fetch the stock prices for AAPL, the current price and any relevant technical indicators please.
"""

result = technical_agent(query)

In [ ]:
# Get just the text response
text_response = result.message['content'][0]['text']
print(text_response)

In [ ]:
# Get metrics if needed
metrics = result.metrics

# Get stop reason
stop_reason = result.stop_reason

# Get token usage
token_usage = result.metrics.accumulated_usage

In [ ]:
print(f"Metrics: {metrics}, stop reason: {stop_reason}, token usage: {token_usage}")

### Market analysis agent
---

Lastly, let's create our market analysis agent. This agent will have access to a few tools to get some option chains and also be able to get some insider trading and news. Since this tool requires more information about insider information, we will add a human in the loop workflow for this agent. In this case, if a user has a question about something that requires or will offer sensitive data, the agent will ask the user for confirmation before using the tool and providing that response back to the user.

In [ ]:
market_analyst_agent_system_prompt: str = """
You are a comprehensive market analysis assistant that helps users analyze options chains, insider trading data, and relevant market news.
You require the user to provide a stock ticker symbol for analysis.
If a user does not provide a ticker symbol, mention in the answer that they need to provide a ticker symbol.

You can perform the following types of analysis. You have access to APIs and tools to call to fetch data based on what the user question is:

1. Use the appropriate functions based on the analysis needed:
    - get_options_chain for options analysis
    - get_insider_trades for insider trading analysis
    - get_news for market news and sentiment

2. Options Chain Analysis:
   - View available options contracts
   - Filter by strike price
   - Filter by option type (call/put)
   - Analyze options pricing and volume

3. Insider Trading Analysis:
   - Recent insider transactions
   - Transaction types (buy/sell)

4. Market News Analysis:
   - Latest relevant news
   - Market sentiment
   - Industry trends

For options chain data, you can specify:
- Strike price filters
- Option type (call/put)
- Number of results to return (limit)

For insider trades, you can specify:
- Number of transactions to analyze (limit)

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above.

Always use the get option chains function first to build some foundational knowledge.
"""

In [ ]:
!uv pip install langgraph

In [ ]:
api_key

In [ ]:
# ------------------------------------------------------------------
# Generic human-approval function
# ------------------------------------------------------------------
def get_human_approval(function_name: str, parameters: dict) -> bool:
    """
    Prompt the human operator via Jupyter notebook input.
    Returns True if approved (human replies "yes"), False otherwise.
    """
    banner = "\n" + "="*60 + "\n🚨  HUMAN APPROVAL REQUIRED  🚨\n" + "="*60
    body = (
        f"{banner}\n"
        f"Function: {function_name}\n\n"
        "Parameters:\n"
        f"{json.dumps(parameters, indent=2)}\n"
        f"{'='*60}"
    )
    
    # Print the approval request
    print(body)
    # Get user input directly in Jupyter notebook
    try:
        answer = input("Do you approve this request? (yes/no): ").strip().lower()
    except KeyboardInterrupt:
        print("\nRequest cancelled by user")
        return False
    except EOFError:
        print("\nNo input received, denying request")
        return False
    
    # Normalize the answer into a boolean
    approved = answer in ("yes", "y", "approve", "true", "1")
    
    if approved:
        print("✅ Request approved")
    else:
        print("❌ Request denied")
    
    return approved

# -----------------------------------
# Tool: get_options_chain (no change)
# -----------------------------------
@tool
def get_options_chain(
    ticker: str,
    limit: int = 10,
    strike_price: Optional[float] = None,
    option_type: Optional[str] = None
) -> Dict:
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    params = {"ticker": ticker, "limit": limit}
    if strike_price is not None:
        params["strike_price"] = strike_price
    if option_type is not None:
        params["option_type"] = option_type

    url = "https://api.financialdatasets.ai/options/chain"
    try:
        resp = requests.get(url, headers={"X-API-Key": api_key}, params=params)
        print(f"RESPONSE FROM THE GET OPTIONS CHAIN TOOL: {resp.json()}")
        return resp.json()
    except Exception as e:
        return {"ticker": ticker, "options_chain": [], "error": str(e)}

# -------------------------------------
# Tool: get_insider_trades (human loop)
# -------------------------------------
@tool
def get_insider_trades(ticker: str, limit: int = 10) -> Dict:
    params = {"ticker": ticker, "limit": limit}

    # always ask the human
    if not get_human_approval("get_insider_trades", params):
        return {
            "error": "Request denied by human operator",
            "function": "get_insider_trades",
            "parameters": params
        }

    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = f"https://api.financialdatasets.ai/insider-transactions?ticker={ticker}&limit={limit}"
    try:
        resp = requests.get(url, headers={"X-API-Key": api_key})
        print(f"RESPONSE FROM THE GET INSIDER TRADES TOOL: {resp.json()}")
        return resp.json()
    except Exception as e:
        return {"ticker": ticker, "insider_transactions": [], "error": str(e)}

# ----------------------------------
# NEW Tool: get_financial_news (using Financial Datasets API)
# ----------------------------------
@tool
def get_news(
    ticker: str, 
    limit: int = 10,
    start_date: Optional[str] = None,
    end_date: Optional[str] = None
) -> Dict:
    """
    Get financial news for a specific ticker using Financial Datasets API.
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'AMZN')
        limit: Number of articles to return (max 100)
        start_date: Start date in YYYY-MM-DD format (optional)
        end_date: End date in YYYY-MM-DD format (optional)
    """
    params = {
        "ticker": ticker, 
        "limit": limit,
        "start_date": start_date,
        "end_date": end_date
    }

    # Remove None values from params
    params = {k: v for k, v in params.items() if v is not None}

    # Ask for human approval
    if not get_human_approval("get_news", params):
        return {
            "error": "Request denied by human operator",
            "function": "get_financial_news",
            "parameters": params
        }

    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    # Build URL with query parameters
    url = "https://api.financialdatasets.ai/news"
    
    try:
        resp = requests.get(
            url, 
            headers={"X-API-KEY": api_key},  # Note: X-API-KEY (not X-API-Key)
            params=params
        )
        print(f"RESPONSE FROM THE GET FINANCIAL NEWS TOOL: {resp.json()}")
        resp.raise_for_status()  # Raise an exception for bad status codes
        return resp.json()
    except Exception as e:
        return {"ticker": ticker, "news": [], "error": str(e)}


In [ ]:
# define the model to be used by the technical analyst agent
market_analysis_agent_info: Dict = config_data['model_information']['market_analysis_agent']

In [ ]:
# create the market analysis agent
market_analysis_agent = Agent(
    model = BedrockModel(
        model_id=market_analysis_agent_info.get('model_id'), 
        temperature=market_analysis_agent_info.get('temperature'), 
        max_tokens=market_analysis_agent_info.get('max_tokens')
    ),
    system_prompt=market_analyst_agent_system_prompt, 
    tools=[get_options_chain, get_news, get_insider_trades]
)

In [ ]:
query: str = """
I want you to get some insider trading information on AAPL, and also some option chains information. If possible, could
you also search on the web for latest news and check if it matches the insider trading information just to be sure if it is
correct/coherent or not? 
"""

In [ ]:
# View the tools in the marketing agent

# We ca see that there are three tools that the agent has access to - 
# getting the option chains, news and insider trades
print_tools_summary(market_analysis_agent.tool_config)

In [ ]:
market_analysis_agent("what tools do you have access to?")

In [ ]:
market_analysis_agent("Can you get me the news for AAPL?")

Now we have successfully added a human in the loop pattern within our tool to review the tool call with the correct parameters and the function to use. If we want to approve that, a human can do so and move to the next step.

### Building a multi-agentic financial system
---

In Multi Agentic systems, it is important to consider what your use case truly aligns to. Are you looking at a use case where you have a task delegation type of a use case, where there are multiple specialized agents or human workers with tasks delegated to them, and then ultimately collating the responses to a main supervisor agent? If so then you might use the supervisor agent architecture. Take an example of the architecture below:

![supervisor-agent-arch](img/supervisor_agent_architecture.png)

In this architecture, we have a supervisor agent that breaks down the task and based on the result, is able to send the task or delegate it to one or more agents. However, let's say we do **not** want to completely hand off control to another agent and rather, have that agent work in a similar way as a tool, then what we can do is that while we retain the supervisor Agentic architecture, we can introduce an agent ***as a tool***. 

In this example, we will convert our three existing agents: 

- Financial agent, 
- Market analyst agent and
- Technical analyst agent

As agent tools that we will attach to a lead_analyst. This lead analyst will decide on which tool to call when and each tool (in this case agents) will have access to their own functionalities, while being tied to the lead analyst Agent.

Strands Agents SDK provides a powerful framework for implementing the "Agents as Tools" pattern through its `@tool` decorator. This allows you to transform specialized agents into callable functions that can be used by an orchestrator agent.

In [ ]:
@tool
def technical_analysis_assistant(query: str) -> str:
    """
    Specialized technical analysis agent for stock charts, indicators, and trading signals.
    
    Use this tool for:
    - Technical indicator analysis (RSI, MACD, Moving Averages, etc.)
    - Chart pattern recognition
    - Support and resistance levels
    - Trading signals and entry/exit points
    - Technical trend analysis
    
    Args:
        query: Technical analysis question or request for specific stock analysis
        
    Returns:
        Detailed technical analysis with indicators, signals, and recommendations
    """
    try:
        # Call your existing technical agent
        response = technical_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in technical analysis: {str(e)}"

@tool
def financial_analysis_assistant(query: str) -> str:
    """
    Specialized financial analysis agent for fundamental analysis and financial metrics.
    
    Use this tool for:
    - Financial statement analysis
    - Valuation metrics (P/E, P/B, PEG ratios, etc.)
    - Revenue and earnings analysis
    - Financial health assessment
    - Comparison with industry peers
    - DCF and other valuation models
    
    Args:
        query: Financial analysis question or request for fundamental analysis
        
    Returns:
        Comprehensive financial analysis with metrics, ratios, and valuation insights
    """
    try:
        # Call your existing finance agent
        response = finance_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in financial analysis: {str(e)}"

@tool
def market_research_assistant(query: str) -> str:
    """
    Specialized market analysis agent for news, sentiment, and market conditions.
    
    Use this tool for:
    - Market news and recent developments
    - Sentiment analysis
    - Sector and industry analysis
    - Market trends and conditions
    - Company news and events
    - Economic indicators impact
    
    Args:
        query: Market research question or request for news/sentiment analysis
        
    Returns:
        Market analysis with news, sentiment, and contextual market information
    """
    try:
        # Call your existing market analysis agent
        response = market_analysis_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in market analysis: {str(e)}"


#### Create the lead analyst agent
---

Now that we have created the three agents with varying tools, let's create a lead analyst agent with these agents as tools.

In [ ]:
LEAD_ANALYST_SYSTEM_PROMPT = """
You are a Lead Financial Analyst that coordinates a team of specialized analysts to provide comprehensive investment analysis.

Your team consists of:

1. **Technical Analysis Assistant** - Use for:
   - Chart analysis and technical indicators
   - Trading signals and entry/exit points
   - Support/resistance levels
   - Technical trend analysis

2. **Financial Analysis Assistant** - Use for:
   - Fundamental analysis and valuation
   - Financial statement analysis
   - Financial ratios and metrics
   - Company financial health

3. **Market Research Assistant** - Use for:
   - Current market news and developments
   - Sentiment analysis
   - Industry and sector analysis
   - Market conditions and trends

**Your Process:**
1. Analyze the user's query to determine which specialists to consult
2. Delegate appropriate aspects to the relevant assistant(s)
3. Synthesize the information from all specialists
4. Provide a comprehensive, well-structured analysis
5. Include clear recommendations based on the combined insights

**Guidelines:**
- For comprehensive stock analysis, consult ALL three assistants
- For specific technical questions, primarily use the Technical Analysis Assistant
- For valuation questions, focus on the Financial Analysis Assistant
- For market context, use the Market Research Assistant
- Always provide a executive summary with clear recommendations
- Present information in a logical, professional format

**Meta-Tooling Capabilities:**
You have access to THREE powerful meta-tooling capabilities via these tools:
- **editor**: Create and modify tool code files
- **load_tool**: Load newly created tools into your toolkit
- **shell**: Execute commands and test tools

Use meta-tooling when users ask for:
- Specialized calculations not available in existing tools (e.g., "Calculate portfolio beta", "Compute Black-Scholes pricing")
- Vague requests that need custom analysis (e.g., "Analyze this stock comprehensively", "What's the best investment strategy?")
- Complex financial models (e.g., Monte Carlo simulations, custom valuation models)
- Data processing tasks (e.g., "Compare these 10 stocks", "Build a sector analysis")

**EXAMPLE TOOLS YOU CAN CREATE:**

1. **Portfolio Beta Calculator Tool**
   - When user asks: "What's the portfolio beta for these stocks?"
   - Create custom_financial_tool_0.py for portfolio beta calculations
   - Accepts: list of tickers, weights, market ticker (default SPY)
   - Returns: Individual betas, weighted portfolio beta, interpretation

2. **Black-Scholes Option Pricing Tool** 
   - When user asks: "What should this option be worth?" or vague option questions
   - Create custom_financial_tool_1.py for options pricing
   - Accepts: stock_price, strike_price, time_to_expiration, risk_free_rate, volatility, option_type
   - Returns: Theoretical option price, Greeks (delta, gamma, theta, vega)

3. **Sector Performance Comparison Tool**
   - When user asks: "How do tech stocks compare?" or broad sector questions  
   - Create custom_financial_tool_2.py for sector analysis
   - Accepts: list of tickers, analysis_period, comparison_metrics
   - Returns: Comparative performance table, rankings, sector insights

**Meta-Tooling Process:**
1. **Identify Need**: When user asks vague questions or needs calculations not in existing tools
2. **Design Tool**: Determine what custom tool is needed
3. **Create Tool**: Use `editor` to write custom_financial_tool_X.py 
4. **Load Tool**: Use `load_tool` to make it immediately available
5. **Use Tool**: Execute the newly created tool
6. **Test**: Use `shell` if needed to verify functionality

**When to Create Custom Tools:**
- User asks vague questions like "Analyze this comprehensively" → Create analysis framework tool
- Specialized calculations needed: "Calculate portfolio beta" → Create portfolio beta tool  
- Custom comparisons: "Compare these 10 stocks" → Create comparison matrix tool
- Complex models: "Run Monte Carlo on my portfolio" → Create simulation tool

**Tool Naming:** Always use "custom_financial_tool_X" format where X is next available number (0, 1, 2, etc.)

**Tool Structure Template:**

Goal:
- Create a python tool under cwd()/tools/*.py using given python tool decorator.
- I have hot-reloading abilities, after writing the file, I can directly use them.
 
Building tools:
 
from strands import tool
 
@tool
def name(name: str, description: str) -> str:
    '''
    Create a tool under cwd()/tools/*.py.
    '''
    return ""

**Your Analysis Process:**
1. Analyze the user's query to determine approach
2. Check if existing specialists can handle the request
3. If not, determine if a custom tool is needed
4. Create custom tools when necessary
5. Delegate to appropriate assistant(s) or use custom tools
6. Synthesize information from all sources
7. Provide comprehensive analysis with clear recommendations

**Guidelines:**
- Always try existing tools first before creating new ones
- For comprehensive analysis, consult ALL three assistants
- Create custom tools for specialized calculations or unique requirements
- Provide executive summary with clear recommendations
- Present information in a logical, professional format

also lastly, you have access to a cron tool to schedule daily analysis for finances and getting the current stock options at 9am.
"""

In [ ]:
print(f"Going to use the {config_data['model_information']['lead_analysis_agent'].get('model_id')} for the lead analyst agent...")

In [ ]:
# define the lead analyst agent
from strands_tools import load_tool, shell, editor, cron

lead_analyst_agent = Agent(
    model=config_data['model_information']['lead_analysis_agent'].get('model_id'),
    system_prompt=LEAD_ANALYST_SYSTEM_PROMPT,
    tools=[
        # Your existing specialist agents as tools
        technical_analysis_assistant,
        financial_analysis_assistant, 
        market_research_assistant,
        # Meta-tooling capabilities - KEY ADDITION!
        load_tool,
        shell, 
        editor, 
        # Scheduling capability - NEW ADDITION!
        cron
    ]
)

In [ ]:
# print the tools that the lead analyst has access to
print_tools_summary(lead_analyst_agent.tool_config)

### Invoke the Multi agentic system
---

In [ ]:
query: str = """
What is the current stock price of AAPL? Are there any particular news related to AAPL that you can provide me with?
Also how are the stocks for AAPL comparing across the years, could you provide a nice journal at the end with all of this analysis?

Tell me all about the market research and the technical analysis you can do with AAPL as well. Can you schedule something too?
"""

In [ ]:
lead_analyst_agent(query)

### Meta tooling
--- 

Now, let's ask it a question for which an agent will create and use a tool on the fly

In [ ]:
lead_analyst_agent("Now, tell me about the comparison or more information about stocks for AAPL across the data that you have. Create a new tool for this.")